In [ ]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
import cv2
import os

import pandas as pd
from tqdm import tqdm

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
# from utils.loss import ComputeLoss

import torch
from torch.utils.data import DataLoader, Dataset

# https://github.com/rwightman/efficientdet-pytorch/tree/master/effdet
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain
from effdet.efficientdet import HeadNet

In [ ]:
class CustomDataset(Dataset):
    '''
      data_dir: data가 존재하는 폴더 경로
      transforms: data transform (resize, crop, Totensor, etc,,,)
    '''

    def __init__(self, annotation, data_dir, transforms=None):
        super().__init__()
        self.data_dir = data_dir
        # coco annotation 불러오기 (coco API)
        self.coco = COCO(annotation)
#         self.image_size = image_size
        self.predictions = {
            "images": self.coco.dataset["images"].copy(),
            "categories": self.coco.dataset["categories"].copy(),
            "annotations": None
        }
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.coco.getImgIds(imgIds=index)
        image_info = self.coco.loadImgs(image_id)[0]
        image = cv2.imread(os.path.join(self.data_dir, image_info['file_name']))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        ann_ids = self.coco.getAnnIds(imgIds=image_info['id'])
        anns = self.coco.loadAnns(ann_ids)

        # boxes (x, y, w, h)
        boxes = np.array([x['bbox'] for x in anns])

        # boxes (x_min, y_min, x_max, y_max)
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        
        # box별 label
#         labels = list([x['category_id'] for x in anns])
        labels = np.array([x['category_id'] for x in anns])
        labels = torch.as_tensor(labels, dtype=torch.int64)
        
        areas = np.array([x['area'] for x in anns])
        areas = torch.as_tensor(areas, dtype=torch.float32)
        
        is_crowds = np.array([x['iscrowd'] for x in anns])
        is_crowds = torch.as_tensor(is_crowds, dtype=torch.int64)
        
        segmentation = np.array([x['segmentation'] for x in anns], dtype=object)

        
        target = {'boxes': boxes, 'labels': labels, 'image_id': torch.tensor([index]), 'area': areas,
                  'iscrowd': is_crowds}

        # transform
        if self.transforms:
            # albumentaion을 거쳐 box가 사라지는 경우를 걸러내기 위해
            while True:
                sample = self.transforms(**{
                    'image': image,
                    'bboxes': target['boxes'],
                    'labels': labels
                })
                if len(sample['bboxes']) > 0:
                    image = sample['image']
                    target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1,0)
                    # efficientdet은 xyxy이 아니라 yxyx 형태로 받기 때문에 변경해줌
                    target['boxes'][:,[0,1,2,3]] = target['boxes'][:,[1,0,3,2]] #yxyx: be warning
                    target['labels'] = torch.tensor(sample['labels'])
                break
                
        return image, target, image_id
            
    def __len__(self) -> int:
        return len(self.coco.getImgIds())

In [ ]:
def get_train_transform():
    return A.Compose([
        A.Resize(512, 512),
        A.Flip(p=0.5),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


def get_valid_transform(image_size):
    return A.Compose([
        A.Resize(image_size, image_size),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [ ]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0


def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
from effdet import DetBenchPredict  #BenchTrain.py 파일에 같이 정의되어 있다,(공식 사이트 gitnub에)
import gc

def load_net(checkpoint_path, device):
    
    # config 파일 불러오기
    config = get_efficientdet_config('tf_efficientdet_d3')
    config.num_classes = 11
    config..images_size = (512,512)
    
    config.soft_nms = False  #nms말고 soft nms를 적용할 것인지
    config.max_det_per_image = 25  # max detection region의 개수
    
    net = EfficientDet(config, pretrained_backbone=True)  #pretrained weight도 같이 load
    net.class_net = HeadNet(config, num_outputs=config.num_classes)  #class개수가 바꿔었기 때문에 새롭게 headNet을 정의해줘야한다.
    
    checkpoint = torch.load(checkpoint_path)
    
    net = DetBenchPredict(net)
    net.load_state_dict(checkpoint)
    net.eval()
    return net.to(device) 

In [ ]:
def valid_fn(val_data_loader, model, device, conf_thres=0.001, iou_thres=0.01):
    outputs = []
    for images, targets, image_ids in tqdm(val_data_loader):
        # gpu 계산을 위해 image.to(device)
        images = torch.stack(images) # bs,ch,w,h - 16,3,512,512
        images = images.to(device).float()
        oytput = model(images)
        for out in output:
            outputs.append({'boxes': out.detach().cpu().numpy()[:,:4],
                            'scores': out.detach().cpu().numpy()[:,:4], 
                            'labels': out.detach().cpu().numpy()[:,-1]})  # 마지막에 label정보 가짐
        

    return outputs

In [ ]:
def main():
    data_dir = '../input/data'
    annotation = '../input/data/val.json'
    val_dataset = CustomDataset(annotation, data_dir, get_valid_transform())
    epoch = 2
    check_point = f'epoch_{epoch}.pth'
    score_threshold = 0.1
    val_data_loader = DataLoader(
        val_dataset,
        batch_size=4,
        shuffle=False,
        num_workers=4,
        collate_fn=collate_fn
    )
    
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print(device)
    
    model = load_net(checkpoint_path, device)
    
    outputs = valid_fn(val_data_loader, model, device)
    
    prediction_strings = []
    file_names = []
    coco = COCO(annotation)
    for i, output in enumerate(outputs):
        prediction_string = ''
        image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
        for box, score, label in zip(output['boxes'], output['scores'], output['labels']):
            if score > score_threshold :
                prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0]) + ' ' + str(
                    box[1]) + ' ' + str(box[2]) + ' ' + str(box[3]) + ' '
        prediction_strings.append(prediction_string)
        file_names.append(image_info['file_name'])
        
    submission = pd.DataFrame()
    submission['PredictionString'] = prediction_strings
    submission['image_id'] = file_names
    submission.to_csv(f'submission_{epoch}.csv', index=None)
    print(submission.head())

In [ ]:
if __name__ == '__main__':
    main()

loading annotations into memory...
Done (t=3.84s)
creating index...
index created!
cuda


100%|██████████| 328/328 [01:03<00:00,  5.18it/s]


loading annotations into memory...
Done (t=4.25s)
creating index...
index created!
                                    PredictionString              image_id
0  1 1.0 94.899994 76.549995 358.5 290.25 8 1.0 1...  batch_01_vt/0003.jpg
1  6 1.0 -169.75 48.449997 313.95 345.15002 2 0.0...  batch_01_vt/0005.jpg
2  6 1.0 -73.600006 -27.84999 339.0 390.05 4 1.0 ...  batch_01_vt/0006.jpg
3  6 1.0 68.69998 -21.25 510.9 165.85 2 0.0021363...  batch_01_vt/0007.jpg
4  1 1.0 -102.15 -46.199997 234.35 325.40002 2 0....  batch_01_vt/0010.jpg
